<a href="https://colab.research.google.com/github/starhou/Algorithm/blob/master/ML/Decision_Tree.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import numpy as np
from sklearn import datasets

[参考](http://blog.lisp4fun.com/2018/03/03/decision-tree)
### 信息增益

> 信息熵：$H=\sum_{i=1}^{n} H\left(x_{i}\right)=-\sum_{i=1}^{n} P\left(x_{i}\right) \log \left(P\left(x_{i}\right)\right)$


> 条件熵：$H(X \| A)=\sum_{i}^{n} p_{i} H\left(X \| A=a_{i}\right)$


> 信息增益：$G = H - H(X \| A)$
*   对特征的取值进行排序（feature 和 target 需要同步排序）
*   将 target 有变动的地方作为可能的划分点（比例，target取值从0变成1）
*   分界点定义为划分点的对应的两个特征取值的平均值
*   计算所有划分点的信息增益，选取最大的那个信息增益对应的值作为最终选择的分界点
*   计算得到所有特征中信息增益最小的特征作为本次所用特征












In [0]:
def calcEntropy(values):
    ''' 根据给定列表中的值计算其Shanno Entropy
    '''
    uniq_vals = np.unique(values)
    P_x = [np.sum(values == val)/values.shape[0] for val in uniq_vals]
    entropy = 0-np.sum(P_x*np.log2(P_x))
    return entropy

In [0]:
def calcConditionEntropy(feature, condition, target):
    true_condition = condition(feature)
    false_condition = true_condition == False
    target_true = target[true_condition]
    target_false = target[false_condition]
    # 每种属性类别的数量除以总数就计算出其概率
    p_true = target_true.size / target.size
    p_false = 1 - p_true
    # 每种属性类别的概率乘以该类别下的信息熵
    entropy = p_true * calcEntropy(target_true) + p_false * calcEntropy(target_false)
    return entropy

In [0]:
def generate_feature_points(feature, target):
    """
    生成特征的所有分界点: 先对特征进行排序，然后将 target 有变动的地方作为分界点
    :param feature: 一维数组，一个特征的样本数据
    :param target: 一维数组，数字或者字符串的分类标签
    :return: 包含所有分界点的一维数组
    """
    argsort = feature.argsort()
    f1 = feature[argsort]
    t1 = target[argsort]

    last_value = t1[0]
    split_value = []

    # 找出所有分裂点
    for i in range(t1.size):
        if last_value != t1[i]:
            split_value.append((f1[i] + f1[i - 1]) / 2)
            last_value = t1[i]

    return np.array(split_value)

In [0]:
def calc_feature_entropy(feature, target):
    """
    计算一个特征的所有分界点的条件熵，返回最小的那个条件熵（条件熵越小，信息增益越大）
    :param feature: 一维数组，一个特征的样本数据
    :param target: 一维数组，数字或者字符串的分类标签
    :return: 分界点和条件熵
    """
    min_entropy = float('inf')
    min_point = 0
    points = generate_feature_points(feature, target)
    for p in points:
        entropy = calcConditionEntropy(feature, lambda f: f < p, target)
        if entropy < min_entropy:
            min_entropy = entropy
            min_point = p

    '没有分界点说明只有一类数据标签，熵为0'
    if points.size == 0:
        min_entropy = 0

    return min_point, min_entropy

In [0]:
def select_feature(features, target):
    """
    从所有特征中选择出条件熵最小的特征（即最大增益）
    :param features: 二维数据，包含所有特征的样本数据
    :param target: 一维数组，数字或者字符串的分类标签
    :return: 特征索引，条件熵，特征分界点
    """
    min_entropy = float('inf')
    min_point = 0
    num = features.shape[1]
    index = 0
    for i in range(num):
        point, entropy = calc_feature_entropy(features[:, i], target)
        if entropy <= min_entropy:
            index = i
            min_point = point
            min_entropy = entropy

    return index, min_point, min_entropy

In [0]:
class TreeNode:
    idn = 0
    feature_index = ''
    feature_point = 0
    feature_entropy = 0
    target_label = ''
    true_node = None
    false_node = None

    @staticmethod
    def decision(feature, point):
        return feature < point

In [0]:
def build_tree(features, target, idn):
    """
    递归构建决策树
    :param features: 二维数据，包含所有特征的样本数据
    :param target: 一维数组，数字或者字符串的分类标签
    :param idn: 决策树节点 id，通过 id 观察决策树计算过程
    :return: 决策树根节点
    """
    node = TreeNode()

    '选择条件熵最小的特征'
    index, point, entropy = select_feature(features, target)
    node.idn = idn
    node.feature_index = index
    node.feature_point = point
    node.feature_entropy = entropy
    '取出现次数最多的标签作为该特征节点的输出'
    print(target)
    node.target_label = target[np.argmax(np.bincount(target))]

    print('build tree node id %d, index %d, point %f, entropy %f, label %s ' %
          (idn, index, point, entropy, node.target_label))

    '熵小于 0.1 时则结束创建子节点，防止过拟合'
    if entropy < 0.1:
        print('too low entropy : ', entropy)
        return node

    f_copy = features.copy()
    t_copy = target.copy()
    f = f_copy[:, index]
    selector = node.decision(f, point)

    '创建左右两个子节点'
    idn = idn + 1
    node.true_node = build_tree(f_copy[selector, :], t_copy[selector], idn)
    idn = node.true_node.idn + 1
    node.false_node = build_tree(f_copy[selector == False], t_copy[selector == False], idn)
    return node

In [0]:
# 4特征，2分类
X, y = datasets.make_classification(n_samples=500, n_features=10, n_informative=2, n_redundant=2)

In [0]:
build_tree(X, y, 1)


信息增益率 Gain Ratio

$(D, a)=\frac{\operatorname{Gain}(D, a)}{I V(a)}$ \
$I V(a)=-\sum_{i}^{n} \frac{\left\|D^{i}\right\|}{\|D\|} \log _{2} \frac{\left\|D^{i}\right\|}{\|D\|}$

基尼指数

$\operatorname{Gini}(D)=1-\sum_{i}^{n} p_{i}^{2}$